# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import googlemaps
import gmaps
import os
import csv

# Import API key
from config import gkey

gmaps.configure(api_key=gkey)
gmaps_places = googlemaps.Client(key=gkey)


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
file = "output_data/cities.csv"
vacation_df = pd.read_csv(file)
vacation_df

,city_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,east london,75,ZA,1578422377,94,-33.02,27.91,69.80,12.75
1,1,cape town,1,ZA,1578422172,78,-33.93,18.42,75.00,10.29
2,2,carnarvon,100,ZA,1578422377,75,-30.97,22.13,66.65,11.99
3,3,mount isa,97,AU,1578422377,62,-20.73,139.49,84.20,4.70
4,4,hobart,1,AU,1578422083,87,-42.88,147.33,57.99,4.47
...,...,...,...,...,...,...,...,...,...,...
558,558,kargil,100,PK,1578422398,95,34.56,76.13,7.09,5.55
559,559,hammerfest,40,NO,1578422398,93,70.66,23.68,37.40,11.41
560,560,comarapa,99,BO,1578422398,72,-17.91,-64.53,72.59,5.21
561,561,taywarah,0,AF,1578422398,75,33.35,64.42,2.93,5.23


100

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
destinations = vacation_df[['Lat', 'Lng']]
humidity_level = vacation_df['Humidity']
fig = gmaps.figure(center =[0,0], zoom_level =2)
fig.add_layer(gmaps.heatmap_layer(destinations, weights=humidity_level[:len(destinations)],max_intensity=250,dissipating=False,
                                 point_radius=5))
fig


Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [4]:

# ideal weather: temperature between 70-80, wind < 10mph, 0 cloudiness
best_weather = vacation_df.loc[(vacation_df["Max Temp"] > 70) & (vacation_df["Max Temp"] < 80)
                           & (vacation_df["Wind Speed"] < 10) & (vacation_df["Cloudiness"] == 0)]
best_weather

,city_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
88,88,ratnagiri,0,IN,1578422380,91,16.98,73.3,70.32,0.85
192,192,rabkavi,0,IN,1578422383,64,16.47,75.1,72.14,5.84
327,327,adre,0,TD,1578422389,26,13.47,22.2,73.22,8.39


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [8]:
hotel_df = best_weather.copy()

hotel_df["Hotel Name"]=""

hotel_df

,city_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
88,88,ratnagiri,0,IN,1578422380,91,16.98,73.3,70.32,0.85,
192,192,rabkavi,0,IN,1578422383,64,16.47,75.1,72.14,5.84,
327,327,adre,0,TD,1578422389,26,13.47,22.2,73.22,8.39,


In [13]:
#setting parameters for hotel search

base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

params = {
    "radius": 5000,
    "keyword": "hotel",
    "key": gkey
}

In [14]:
for index, row in hotel_df.iterrows():
    lat = row["Lat"]
    lng = row["Lng"]
    city = row["City"]
    params["location"] = f"{lat}, {lng}"
    response = requests.get(base_url, params=params).json()
    # create exception if no hotel is found
    try:
        results = response["results"][0]["name"]
        hotel_df.loc[index, "Hotel Name"] = results
    except (IndexError):
        hotel_df.loc[index, "Hotel Name"] = "N/A"
        print(f"No hotel found in {city}. Skipping...")
    
hotel_df


No hotel found in rabkavi. Skipping...
No hotel found in adre. Skipping...


,city_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
88,88,ratnagiri,0,IN,1578422380,91,16.98,73.3,70.32,0.85,Hotel Sea Fans
192,192,rabkavi,0,IN,1578422383,64,16.47,75.1,72.14,5.84,N/A
327,327,adre,0,TD,1578422389,26,13.47,22.2,73.22,8.39,N/A


In [15]:
#remove rows with no hotel

hotel_df2 = hotel_df[(hotel_df != "N/A").all(axis=1)]
    
hotel_df2

C:\Users\Rand\anaconda\Anaconda3\lib\site-packages\pandas\core\ops\__init__.py:1115: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = method(y)


,city_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
88,88,ratnagiri,0,IN,1578422380,91,16.98,73.3,70.32,0.85,Hotel Sea Fans


In [16]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df2.iterrows()]
locations = hotel_df2[["Lat", "Lng"]]

In [17]:
# Add marker layer ontop of heat map
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# Display Map
fig

Figure(layout=FigureLayout(height='420px'))